In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scraping_class
from matplotlib.dates import DateFormatter
import seaborn as sns
from bs4 import BeautifulSoup

In [2]:
plt.style.use('ggplot')
%matplotlib inline

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
logfile = 'oscar_winners_log.csv'
connector = scraping_class.Connector(logfile)

In [222]:

movies_data = []

for i in range(1,100, 100):
    #url_nominees = 'https://www.imdb.com/search/title/?groups=oscar_best_picture_nominees&start=%s&ref_=adv_nxt' % str(i)
    url_winners = 'https://www.imdb.com/search/title/?count=100&groups=oscar_best_picture_winners&sort=year,desc&start=%s&ref_=nv_ch_osc' % str(i)

    response, call_id = connector.get(url_winners, 'get_winners')
    if response.ok:
        data = response.text
        soup = BeautifulSoup(data, "lxml")
        movies = soup.find_all('div', attrs={'lister-item-content'})


        for movie in movies:
            
            try:
                runtime = movie.find('p', attrs={'text-muted'}).find('span', attrs={'runtime'}).text
                genre   = movie.find('p', attrs={'text-muted'}).find('span', attrs={'genre'}).text.strip().split(', ')
                    
                
                idx, title, year = movie.find('h3', attrs={'lister-item-header'}).text.strip().split('\n')
                idx = idx.split('.')[0]
                link = movie.find('a')
                link = str(link).split('href="')[1].split('"')[0]
                link  = 'https://www.imdb.com' + link
                
                try:
                    if movie.find_all('span')[-2].text == 'Gross:':
                        gross = movie.find_all('span', attrs={'name':'nv'})[-1].text
                    else:
                        gross = np.nan
                except:
                    gross = np.nan
                
                try:
                    metascore = movie.find('span', attrs={'metascore'}).text
                except:
                    metascore = np.nan
                    
                job = movie.find_all('p')[2].text
                
                actors = [i.split('\n')[1] for i in job.split('Stars:')[1].split(',')]
                directors = [i.split(',')[0] for i in job.split('Stars:')[0].split('Director')[1].split(':\n')[1].split('\n')[:-2]]

                try:
                    year = year.split('(I')[1]
                    try:
                        year = year.split('I) (')[1].split(')')[0]
                    except:
                        year = year.split(') (')[1].split(')')[0]
                except:
                    year = year.split('(')[1].split(')')[0]
                    
                movies_data.append([idx, title, gross, link, year, directors, actors, runtime, genre, metascore])
                
            except:
                NameError
            
    else:
        print('Response failed!')
        
df = pd.DataFrame(movies_data)
df.columns = ['index', 'title', 'gross', 'link', 'year', 'director', 'actors', 'runtime', 'genre', 'metascore']

In [224]:
df.set_index('index')

,title,gross,link,year,director,actors,runtime,genre,metascore
index,,,,,,,,,
1,Green Book,$85.08M,https://www.imdb.com/title/tt6966692/,2018,[Peter Farrelly],"[Viggo Mortensen, Mahershala Ali, Linda Cardel...",130 min,"[Biography, Comedy, Drama]",69
2,The Shape of Water,$63.86M,https://www.imdb.com/title/tt5580390/,2017,[Guillermo del Toro],"[Sally Hawkins, Octavia Spencer, Michael Shann...",123 min,"[Adventure, Drama, Fantasy]",87
3,Moonlight,$27.85M,https://www.imdb.com/title/tt4975722/,2016,[Barry Jenkins],"[Mahershala Ali, Naomie Harris, Trevante Rhode...",111 min,[Drama],99
4,Spotlight,$45.06M,https://www.imdb.com/title/tt1895587/,2015,[Tom McCarthy],"[Mark Ruffalo, Michael Keaton, Rachel McAdams,...",129 min,"[Biography, Crime, Drama]",93
5,Birdman or (The Unexpected Virtue of Ignorance),$42.34M,https://www.imdb.com/title/tt2562232/,2014,[Alejandro G. Iñárritu],"[Michael Keaton, Zach Galifianakis, Edward Nor...",119 min,"[Comedy, Drama]",87
6,12 Years a Slave,$56.67M,https://www.imdb.com/title/tt2024544/,2013,[Steve McQueen],"[Chiwetel Ejiofor, Michael Kenneth Williams, M...",134 min,"[Biography, Drama, History]",96
7,Operation Argo,$136.03M,https://www.imdb.com/title/tt1024648/,2012,[Ben Affleck],"[Ben Affleck, Bryan Cranston, John Goodman, Al...",120 min,"[Biography, Drama, Thriller]",86
8,The Artist,$44.67M,https://www.imdb.com/title/tt1655442/,2011,[Michel Hazanavicius],"[Jean Dujardin, Bérénice Bejo, John Goodman, J...",100 min,"[Comedy, Drama, Romance]",89
9,Kongens store tale,$138.80M,https://www.imdb.com/title/tt1504320/,2010,[Tom Hooper],"[Colin Firth, Geoffrey Rush, Helena Bonham Car...",118 min,"[Biography, Drama, History]",88


In [221]:
df.to_csv('oscar_winners.csv')